In [3]:
import zmq
context = zmq.Context()

print("Connecting to Client…")
client_socket = context.socket(zmq.PULL)
client_socket.bind("tcp://*:5555")

while True:
    message = client_socket.recv()
    print('Message received')

Connecting to Client…
Message received
Message received
Message received
Message received
Message received
Message received
Message received
Message received
Message received
Message received
Message received
Message received


KeyboardInterrupt: ignored

In [ ]:
import time
import zmq

context = zmq.Context()
socket = context.socket(zmq.PULL)
socket.bind("tcp://*:5555")

while True:
    #  Wait for next request from client
    message = socket.recv()
    print(f"Received request:")

    #  Do some 'work'
    #time.sleep(1)

    #  Send reply back to client
    #socket.send(b"World")

Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received request:
Received r